# More Capable Functions

N-arity and variadic functions

In [1]:
;; A simple function that takes a variable number of args
;; (a multi-arity function)

(defn greet
    ([to-whom] 
        (println "Welcome to Blotts Books" to-whom))
    ([message to-whom] 
        (println message to-whom)))


#'user/greet

In [2]:
(greet "Dolly")


Welcome to Blotts Books Dolly


nil

In [3]:
(greet "Howdy" "Stranger")


Howdy Stranger


nil

In [4]:
;; One of the arities of the function can be expressed in terms of the other

(defn greet
    ([to-whom]
        (greet "Welcome to Blotts Books" to-whom)) ; Expressed in terms of the other arity
    ([message to-whom] 
        (println message to-whom)))


#'user/greet

In [8]:
;; A function that takes an arbitrary number of args
;; (a variadic function)

(defn print-any-args [& args] ; The '&' symbol means a variable number of args
    (println "My arguments are:" args))

(print-any-args 7 true nil) ; The args are printed as a collection  


My arguments are: (7 true nil)


nil

In [10]:
;; Functions can have ordinary args before '&', as follows

(defn first-argument [& args]
    (first args))

(defn new-first-argument [x & args] 
    x)


#'user/new-first-argument

In [11]:
(first-argument 1 2 3)

1

In [12]:
(new-first-argument 1 2 3)

1

Multimethods

In [17]:
;; Given the following formats of book data

{:title "War and Peace" :author "Tolstoy"}

{:book "Emma" :by "Austen"}

["1984" "Orwell"]


["1984" "Orwell"]

In [19]:
;; A function that normalizes the format of book 
;; can be written as follows
;; (It works, but with a growing number of book formats it will get ugly fast)

(defn normalize-book [book]
    (if (vector? book)
        {:title (first book) :author (second book)}
        (if (contains? book :title)
            book
            {:title (:book book) :author (:by book)})))


#'user/normalize-book

In [20]:
(normalize-book {:title "War and Peace" :author "Tolstoy"})


{:title "War and Peace", :author "Tolstoy"}

In [21]:
(normalize-book {:book "Emma" :by "Austen"})


{:title "Emma", :author "Austen"}

In [22]:
(normalize-book ["1984" "Orwell"])


{:title "1984", :author "Orwell"}

In [27]:
;; A function that returns keywords based on the book's format
;; to be used later to define a multimethod

(defn dispatch-book-format [book]
    (cond
        (vector? book) :vector-book
        (contains? book :title) :standard-map
        (contains? book :book) :alternative-map))

(dispatch-book-format ["1984" "Orwell"])


:vector-book

In [26]:
;; Defining a multimethod from the last dispatcher function

(defmulti normalize-book dispatch-book-format)


nil

In [35]:
;; Implementing the last multimethod for the possible values
;; returned from the dispatch function

(defmethod normalize-book :vector-book [book]
    {:title (first book) :author (second book)})

(normalize-book ["1984" "Orwell"])


{:title "1984", :author "Orwell"}

In [36]:
(defmethod normalize-book :standard-map [book]
    book)

(normalize-book {:title "War and Peace" :author "Tolstoy"})


{:title "War and Peace", :author "Tolstoy"}

In [34]:
(defmethod normalize-book :alternative-map [book]
    {:title (:book book) :author (:by book)})

(normalize-book {:book "Emma" :by "Austen"})


{:title "Emma", :author "Austen"}

In [46]:
;; A new colection of books with a new keyword :genre,
;; which should be processed somehow by the last multimethod

(def books [{:title "Pride and Prejudice" :author "Austen" :genre :romance}
            {:title "World War Z" :author "Brooks" :genre :zombie}])


#'user/books

In [50]:
;; In case another keyword is present (:genre in this case), 
;; a separate multimethod can be created independently, as follows

(defmulti book-description :genre)

(defmethod book-description :romance [book]
    (str "The heart warming new romance by " (:author book)))

(defmethod book-description :zombie [book]
    (str "The heart consuming new zombie adventure by " (:author book)))


#multifn[book-description 0x73ecc2fe]

In [51]:
(book-description (first books))

"The heart warming new romance by Austen"

In [52]:
(book-description (last books))

"The heart consuming new zombie adventure by Brooks"

In [47]:
;; If there are new genres, a new method can be implemented
;; to handle it, as follows

(def ppz {:title "Pride and Prejudice and Zombies"
          :author "Grahame-Smith"
          :genre :zombie-romance})

(defmethod book-description :zombie-romance [book]
    (str "The heart warming and consuming new romance by " (:author book)))

(book-description ppz)

"The heart warming and consuming new romance by Grahame-Smith"

Recursive functions

In [53]:
;; Given the following books map

(def books
    [{:title "Jaws" :copies-sold 2000000}
     {:title "Emma" :copies-sold 3000000}
     {:title "2001" :copies-sold 4000000}])


#'user/books

In [54]:
;; To get the sum of the copies sold, 
;; a recursive function can be defined as follows

(defn sum-copies
    ([books] 
        (sum-copies books 0))
    ([books total]
        (if (empty? books)
        total
        (sum-copies ; Note the recursion here
            (rest books)
            (+ total (:copies-sold (first books)))))))

(sum-copies books)


9000000

In [59]:
;; The last function will blow the stack with a large collection.
;; To avoid that, the 'recur' function can be used as follows

(defn sum-copies
    ([books] (sum-copies books 0))
    ([books total]
        (if (empty? books)
            total
            (recur
                (rest books)
                (+ total (:copies-sold (first books)))))))

(sum-copies books)

9000000

In [61]:
;; The last function can be made even shorter
;; with the 'loop' function, as follows

(defn sum-copies [books]
    (loop [books books total 0]
        (if (empty? books)
            total
            (recur (rest books) (+ total (:copies-sold (first books)))))))

(sum-copies books)


9000000

Docstrings

In [67]:
"""
To describe a function's purpose, it's ok to describe it with standard comments
"""

;; Return the average of the two parameters.

(defn average [a b]
    (/ (+ a b) 2.0))

(average 4 3)

3.5

In [71]:
;; But a more idiomatic way is to use docstrings, as follows

(defn average-2 
    "Return the average of a and b."
    [a b]
    (/ (+ a b) 2.0))

(average-2 4 3)


3.5

Pre and Post-conditions

In [86]:
;; In case you want to check some property of the args (a map, in this case)
;; a conditional checking can be written at the start of the function, as follows

(defn publish-book [book]
    (when-not (contains? book :title)
        (throw (ex-info "Books must contain :title" {:book book})))
    (println book))


#'user/publish-book

In [85]:
(publish-book {:title "War and Peace" :author "Tolstoy"}) ; Pass the checking

{:title War and Peace, :author Tolstoy}


nil

In [100]:
(publish-book {:author "Tolstoy"}) ; Doesn't pass the checking

Execution error (AssertionError) at user/publish-book (REPL:1).
Assert failed: (:title book)


class java.lang.AssertionError: 

In [91]:
;; But Clojure provides a keyword for the last functionality, 
;; the :pre (for pre-conditional) keyword

(defn publish-book-2 [book]
    {:pre [(:title book)]} ; The pre-conditional should be a vector of expressions
    (println book))


#'user/publish-book-2

In [92]:
(publish-book-2 {:title "War and Peace" :author "Tolstoy"}) ; Pass the pre-condition

{:title War and Peace, :author Tolstoy}


nil

In [93]:
(publish-book-2 {:author "Tolstoy"}) ; Doesn't pass the pre-condition

Execution error (AssertionError) at user/publish-book-2 (REPL:1).
Assert failed: (:title book)


class java.lang.AssertionError: 

In [98]:
;; A similar functionality but for return values is present 
;; in the :post (for post-conditional) keyword


(defn publish-book-3 [book]
    {:pre [(:title book) (:author book)]
     :post [(boolean? %)]} ; The post-conditional should be a vector of expressions
    (map? book))


#'user/publish-book-3

In [99]:
(publish-book-3 {:title "War and Peace" :author "Tolstoy"})

true

Issues with functions

In [102]:
;; Trying to define a n-ary function with overlapping args throws an exception

(defn one-two-or-more
    ([a] (println "One arg:" a))
    ; It's not clear which of the 2 below should be called when there are 2 args
    ([a b] (println "Two args:" a b))
    ([& more] (println "More than two:" more))) 


Syntax error compiling fn* at (REPL:3:1).
Can't have fixed arity function with more params than variadic function


class clojure.lang.Compiler$CompilerException: 

In [103]:
;; To avoid ambiguities, the last function should be modified as follows

(defn one-two-or-more
    ([a] (println "One arg:" a))
    ([a b] (println "Two args:" a b))
    ([a b & more] (println "More than two:" a b more)))


#'user/one-two-or-more

In [105]:
;; Functions with multiple body expressions should not be confused
;; with multi-arity functions

(defn chatty-average
    ([a b]
        (println "chatty-average function called with 2 arguments")
        (println "** first argument:" a)
        (println "** second argument:" b)
        (/ (+ a b) 2.0)))

(defn chatty-multi-average
    ([a b]
        (println "chatty-average function called with 2 arguments")
        (/ (+ a b) 2.0))
    ([a b c]
        (println "chatty-average function called with 3 arguments")
        (/ (+ a b c) 3.0)))


#'user/chatty-multi-average

In [107]:
(chatty-average 2 3)

chatty-average function called with 2 arguments
** first argument: 2
** second argument: 3


2.5

In [108]:
(chatty-multi-average 2 3)

chatty-average function called with 2 arguments


2.5